# BERT Embedding

This note book is used to generate BERT embeddings for each book to be used as the item feature vector.


In [ ]:
# NOTE: this cell is for defining the paths and other configurations
# TODO: change it based on your environment!

colab = True

if colab:
    from google.colab import drive

    drive.mount("/content/drive")
    save_path = "/content/drive/MyDrive/"
else:
    save_path = "../../data/embeddings/"

if colab:
    data_path = "metadata_clean.csv"
else:
    data_path = "../../data/metadata/metadata_clean.csv"

Mounted at /content/drive


In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
import numpy as np

In [ ]:
df = pd.read_csv(data_path)
df = df[["ISBN", "description"]]
df.dropna(inplace=True)

## Model

We use microsoft/deberta-v3-base as SOTA from huggingface.

In [ ]:
model_name = "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"using device: {device}")

using device: cuda


## Embedding

We save both CLS and mean of all token embeddings as the item feature vector. We do the embedding process in batches to fasten up the process.


In [ ]:
batch_size = 32
isbn_list = df["ISBN"].tolist()
descriptions = df["description"].tolist()

cls_embeddings = []
mean_embeddings = []

model.eval()
with torch.no_grad():
    for i in tqdm(
        range(0, len(descriptions), batch_size), desc="Encoding descriptions"
    ):
        batch_desc = descriptions[i : i + batch_size]

        inputs = tokenizer(
            batch_desc,
            return_tensors="pt",
            add_special_tokens=True,
            truncation=True,
            padding="max_length",
            max_length=512,
        )

        inputs = {k: v.to(device) for k, v in inputs.items()}
        outputs = model(**inputs)
        cls_emb = outputs.last_hidden_state[:, 0, :]

        mask = (
            inputs["attention_mask"]
            .unsqueeze(-1)
            .expand(outputs.last_hidden_state.size())
            .float()
        )
        sum_emb = torch.sum(outputs.last_hidden_state * mask, dim=1)
        token_count = torch.clamp(mask.sum(dim=1), min=1e-9)
        mean_emb = sum_emb / token_count

        cls_embeddings.append(cls_emb.cpu().numpy())
        mean_embeddings.append(mean_emb.cpu().numpy())

cls_embeddings = np.concatenate(cls_embeddings, axis=0)
mean_embeddings = np.concatenate(mean_embeddings, axis=0)

with open(save_path + "isbn_list.txt", "w") as f:
    f.write("\n".join(isbn_list))
np.save(save_path + "cls_embeddings.npy", cls_embeddings)
np.save(save_path + "mean_embeddings.npy", mean_embeddings)

print(cls_embeddings.shape)
print(mean_embeddings.shape)

Encoding descriptions: 100%|██████████| 509/509 [12:59<00:00,  1.53s/it]


(16261, 768)
(16261, 768)
